In [34]:
from selenium import webdriver
import time
import pandas as pd
import pickle
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

driver = webdriver.Chrome('./chromedriver')  # Optional argument, if not specified will search path.

In [4]:
driver.get('https://www.cityofboston.gov/cityclerk/dbasearch/')

In [5]:
li = driver.find_element_by_id('business_neighborhood')

In [9]:
nbrs = []
for i in li.find_elements_by_tag_name('option'):
    nbrs.append(i.text)
nbrs = nbrs[1:]

In [35]:
keys_name = ['Hospital','Health','Medical','Clinic','Nursing','Physician']
keys_type = ['Hospital','Health','Healthcare','Medical','Patient','Clinic','Nurse','Physician']
out = []
seen_files = []
for i in keys_type:
    time.sleep(1)
    driver.get('https://www.cityofboston.gov/cityclerk/dbasearch/Default.aspx?type_of_business=' + i)
    btn = driver.find_element_by_xpath('//input[@title="Next Page"]')
    while btn.get_attribute("onclick") != "return false;":
        tbl = driver.find_element_by_id('RadGrid1_ctl00__0')
        rows = tbl.find_elements_by_tag_name('div')
        for row in rows:
            t = row.text.split('\n')
            out.append(t)
            seen_files.append(t[1].split(': ')[1])
        time.sleep(1)
        btn.click()
        btn = driver.find_element_by_xpath('//input[@title="Next Page"]')
for i in keys_name:
    time.sleep(1)
    driver.get('https://www.cityofboston.gov/cityclerk/dbasearch/Default.aspx?business_name=' + i)
    btn = driver.find_element_by_xpath('//input[@title="Next Page"]')
    while btn.get_attribute("onclick") != "return false;":
        tbl = driver.find_element_by_id('RadGrid1_ctl00__0')
        rows = tbl.find_elements_by_tag_name('div')
        for row in rows:
            t = row.text.split('\n')
#             try:
#                 business_type = t[5].split(': ')[1]
#             except:
#                 business_type = ""
#             try:
#                 business_name = t[0].split(': ')[1]
#             except:
#                 business_name = ""
#             for key in keys:
#                 if key in business_type or key in business_name:
            if t[1].split(': ')[1] not in seen_files:
                out.append(t)
        time.sleep(1)
        btn.click()
        btn = driver.find_element_by_xpath('//input[@title="Next Page"]')

In [38]:
import csv
cs = open('boston_businesses.csv', 'w')
writer = csv.writer(cs, delimiter=',')
print(out[0])

['Business Name: CuraHealth Boston', 'File Number: CC310528', 'Issue Date: 12/6/2016', 'Last Renewal Date:', 'Expiration Date: 12/6/2020', 'Type of Business: Hospital', 'Business Address: 1515 Commonwealth AVE Brighton MA 02135', 'Owner Name: Curahealth Boston, LLC', 'Owner Address: 650 Beebalm LN, Suite 220', 'Owner Email: ken.mcgee@curahealth.com', ' ']


In [39]:
columns = []
for i in out[0]:
    columns.append(i.split(': ')[0])

In [42]:
writer.writerow(columns)
for i in out:
    arr = []
    for line in i:
        try:
            arr.append(line.split(': ')[1])
        except:
            arr.append('')
    writer.writerow(arr)
cs.close()